# 알고리즘 및 실습 실습과제 11

In [33]:
import random

WALL_CHARACTER = '#';
# 벽을 표현하는 문자
PATH_CHARACTER = ' ';


# 빈 통로를 표현하는 문자


# #######
# #N#N#N#
# #######
# #N#N#N#
# #######
# 위의 미로처럼 외벽 내부에 노드 지점을 비운 미로 2차원 배열을 반환
def generate_maze_frame(width, height):
    maze = [[WALL_CHARACTER] * width for _ in range(height)]
    for h in range(1, height - 1, 2):
        for w in range(1, width - 1, 2):
            maze[h][w] = PATH_CHARACTER
    return maze


# 미로 생성을 위한 좌표-속성 사전 생성
# (Y, X) = 속성 식으로 생성하며
# 속성은 노드, 경로의 좌표에 대해서 생성되며 사용자가 인자로 넘겨준 initializer의 반환 값으로 생성됨
def generate_maze_properties(width, height, initializer):
    properties = {}
    for h in range(1, height - 1, 2):
        for w in range(1, width - 1, 2):
            properties[(h, w)] = initializer("node", width, height, w, h)
    # 노드 부분의 속성 설정
    for h in range(1, height - 1, 2):
        for w in range(1 + 1, width - 1, 2):
            properties[(h, w)] = initializer("horizontal_path", width, height, w, h)
    # 가로 통로에 대한 속성 설정
    for h in range(2, height - 1, 2):
        for w in range(1, width, 2):
            properties[(h, w)] = initializer("vertical_path", width, height, w, h)
    # 세로 통로에 대한 속성 설정
    return properties


# 미로를 스트링으로 변환하는 함수
def stringify(maze):
    return '\n'.join([''.join(w) for w in maze])


# 현재 위치, 현재 위치의 속성, 미로 크기를 인자로 받아 해당 위치의 속성 사전을 생성하는 함수
# near: 인근의 통로와 노드의 위치를 담은 리스트
# type: node, horizontal_path, vertical_path가 될 수 있고 해당 위치의 형태를 반환
# weight: path 형태만 가질 수 있으며, 랜덤으로 생성한 가중치가 부여됨
def generate_property(type, width, height, x, y):
    prop = {"type": type, "pos": (y, x)}
    if type == "node":
        prop["near"] = list(filter(lambda c: 1 <= c[0] < height - 1 and 1 <= c[1] < width - 1,
                                   [(y, x + 1), (y + 1, x), (y, x - 1), (y - 1, x)]))
    else:
        prop["weight"] = random.random()
        if type == "horizontal_path":
            prop["near"] = [(y, x - 1), (y, x + 1)]
        elif type == "vertical_path":
            prop["near"] = [(y - 1, x), (y + 1, x)]
    return prop


# 최소 신장 트리를 이용하여 미로를 만드는 함수
def generate_maze(width, height):
    maze = generate_maze_frame(width, height)
    # 미로의 기본 골격 생성
    properties = generate_maze_properties(width, height, generate_property)
    # 미로 생성에 필요한 좌표-속성 사전
    prop_list = list(properties.values())
    # 전체 속성 리스트
    nodes = list(filter(lambda prop: prop["type"] == "node", prop_list))
    # 노드 속성만 모은 리스트

    current_node = nodes[0]
    # 현재 노드를 가장 첫번째 노드로 생성함
    visited = [current_node]
    # 방문한 노드를 기록하는 리스트
    look_forward = 0

    number_of_nodes = len(nodes)
    while number_of_nodes != len(visited):
        # 방문한 노드가, 전체 노드 수가 될 때까지 반복
        target_paths = sorted(map(lambda pos: properties[pos], current_node["near"]), key=lambda path: path["weight"],
                              reverse=True)
        # 현재 노드의 인근 경로에서 가중치를 기준으로 오름차순 정렬한 인근 경로 속성 리스트를 가져옴
        lowest_weight_path = None
        next_node = None

        while True:
            if target_paths:
                # 현재 노드의 인근 경로 리스트에 요소가 존재하는 경우
                lowest_weight_path = target_paths.pop()
                # 최소 가중치 경로 속성 가져오기
                (next_pos,) = set(lowest_weight_path["near"]) - set([current_node["pos"]])
                # 최소 가중치 경로로 이동했을 때의 노드 위치를 가져오기
                if properties[next_pos] not in visited:
                    next_node = properties[next_pos]
                    # 가져온 노드가 이미 방문한 노드가 아니면 next_node를 초기화
                    break
            else:
                break

        if next_node:
            # next_node가 있는 경우
            (y, x) = lowest_weight_path["pos"]
            maze[y][x] = PATH_CHARACTER
            # 미로에서 해당 이동 경로의 위치를 PATH_CHARACTER로 변경

            visited.append(next_node)
            current_node = next_node
            # 현재 노드로 구한 다음 노드를 설정하고, visited 노드에 추가
            look_forward = 0
            # look_forward 커서 초기화
        else:
            current_node = visited[look_forward]
            look_forward -= 1
            # next_node를 구하지 못한 경우 방문 요소를 되돌아가면서 위의 과정 반복
    return maze


print(stringify(generate_maze(27, 17)))
print(stringify(generate_maze(29, 19)))
print(stringify(generate_maze(31, 21)))
# 각 사이즈별 미로 생성

###########################
#   #     #   #     # #   #
# ### ### # # # ### # # # #
#   #   #   #   # # # # # #
### ### ######### # # # # #
#   #   #   #       #   # #
# ### ### # # ######### # #
# #   #   # # #       # # #
# # ##### # # # ##### ### #
#   #     # # #   # #   # #
####### # ### ### # ### # #
#       # #   #   # #   # #
# ### ### # ### ### # ### #
# #   #   # #   #   # #   #
# # ####### # ### # # # # #
# #           #   #     # #
###########################
#############################
#   # #   #           #     #
### # # # # ##### ### ### # #
#   #   #   #   # #       # #
# ### ######### # ######### #
#   #       #   # #       # #
### ####### # ### # ##### # #
# # #   #   #   #   #   # # #
# # # # ### ### ####### # ###
# # # #   #             #   #
# # # ### ############# ### #
# # #   #   #     #     # # #
# # ### ### # ### # ##### # #
# #   #   # # # # #   #   # #
# ### ### # # # # ### # ### #
#   #     # # #   #     #   #
# # ####### # # ######### # #
# #           